In [2]:
import os
import urllib

from osgeo import gdal
from math import floor, ceil
from pyproj import Proj
import matplotlib.pyplot as plt
import xarray as xr
import numpy as np
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
import cartopy.crs as ccrs
import dask
import glob
import time
import xesmf as xe
# import quantile_mapping as qm


import sys
# sys.path.append('/glade/u/home/bkruyt/libraries/storylines/storylines')
sys.path.append('/glade/u/home/bkruyt/libraries')
sys.path.append('/glade/u/home/bkruyt/libraries/st_lines')
sys.path
import icar2gmet as i2g
from storylines.tools import quantile_mapping as qm
### If storylines gives an error , use py3_yifan kernel /conda env
# alternatively ../Example_scripts/2D_quantile_map/quantile_mapping.py

In [3]:

def get_latlon_b(ds,lon_str,lat_str,lon_dim,lat_dim):
    #### Longitude East-West Stagger
    diffWestEast    = ds[lon_str].diff(lon_dim)                         # take difference in longitudes across west-east (columns) dimension
    diffWestEast    = np.array(diffWestEast)                            # assign this to numpy array
    padding         = diffWestEast[:,-1].reshape(len(diffWestEast[:,-1]),1)  # get the last column of the differences and reshape so it can be appended
    diffWestEastAll = np.append(diffWestEast, padding, axis=1)/2               # append last value to all_data

    # dimensions now the same as ds
    lon_b_orig      = ds[lon_str]-diffWestEastAll

    # lon_b needs to be staggered - add to the final row to bound
    last_add     = ds[lon_str][:,-1]+diffWestEastAll[:,-1]
    last_add     = np.array(last_add)
    last_add     = last_add.reshape(len(last_add),1)
    lon_b_append = np.append(np.array(lon_b_orig),last_add,1)
    last_add     = lon_b_append[0,:].reshape(1,len(lon_b_append[0,:]))
    lon_b_append = np.append(last_add,lon_b_append,axis=0)


    #### Latitude Stagger
    diffSouthNorth=ds[lat_str].diff(lat_dim)                         # take difference in longitudes across west-east (columns) dimension
    diffSouthNorth=np.array(diffSouthNorth)                            # assign this to numpy array
    padding=diffSouthNorth[0,:].reshape(1,len(diffSouthNorth[0,:]))  # get the last column of the differences and reshape so it can be appended
    diffSouthNorthAll = np.append(padding,diffSouthNorth,axis=0)/2               # append last value to all_data

    # dimensions now the same as ds
    lat_b_orig      = ds[lat_str]-diffSouthNorthAll

    # lat_b needs to be staggered - add to the first row to bound
    last_add     = ds[lat_str][0,:]+diffWestEastAll[0,:]
    last_add     = np.array(last_add)
    last_add     = last_add.reshape(1,len(last_add))
    lat_b_append = np.append(last_add,np.array(lat_b_orig),axis=0)
    last_add     = lat_b_append[:,-1].reshape(len(lat_b_append[:,-1]),1)
    lat_b_append = np.append(lat_b_append,last_add,axis=1)

    
    grid_with_bounds = {'lon': ds[lon_str].values,
                               'lat': ds[lat_str].values,
                               'lon_b': lon_b_append,
                               'lat_b': lat_b_append,
                              }

    return grid_with_bounds

def get_latlon_b_rect(ds,lon_str,lat_str,lon_dim,lat_dim):
    #### Longitude Stagger
    diffWestEast    = ds[lon_str].diff(lon_dim)                         # take difference in longitudes across west-east (columns) dimension
    diffWestEastArr = np.array(diffWestEast).reshape(len(diffWestEast),1)                            # assign this to numpy array
    padding         = diffWestEastArr[-1].reshape(1,1)  # get the last column of the differences and reshape so it can be appended
    diffWestEastAll = np.append(diffWestEastArr, padding, axis=0)/2               # append last value to all_data
    # # dimensions now the same as ds
    lon_b_orig      = ds[lon_str].values.reshape(len(ds[lon_str]),1)-diffWestEastAll

    # lon_b needs to be staggered - add to the final row to bound
    last_add        = ds[lon_str][-1].values+diffWestEastAll[-1]
    last_add        = np.array(last_add).reshape(len(last_add),1)
    last_add        = last_add.reshape(1,1)
    lon_b_append    = np.append(np.array(lon_b_orig),last_add,0)

    #### Latitude Stagger
    diffSouthNorth    = ds[lat_str].diff(lat_dim)                         # take difference in latitudes across west-east (columns) dimension
    diffSouthNorthArr = np.array(diffSouthNorth).reshape(len(diffSouthNorth),1)                            # assign this to numpy array
    padding         = diffSouthNorthArr[-1].reshape(1,1)  # get the last column of the differences and reshape so it can be appended
    diffSouthNorthAll = np.append(diffSouthNorthArr, padding, axis=0)/2               # append last value to all_data
    # # dimensions now the same as ds
    lat_b_orig      = ds[lat_str].values.reshape(len(ds[lat_str]),1)-diffSouthNorthAll

    # lat_b needs to be staggered - add to the final row to bound
    last_add        = ds[lat_str][-1].values+diffSouthNorthAll[-1]
    last_add        = np.array(last_add).reshape(len(last_add),1)
    last_add        = last_add.reshape(1,1)
    lat_b_append    = np.append(np.array(lat_b_orig),last_add,0)

    grid_with_bounds = {'lon': ds[lon_str],
                               'lat': ds[lat_str],
                               'lon_b': lon_b_append.reshape(len(lon_b_append),),
                               'lat_b': lat_b_append.reshape(len(lat_b_append),),
                              }
    return grid_with_bounds


In [21]:
### SET paths and parameters  ####

# ERAi
# era_path='/glade/u/home/currierw/scratch/erai/convection/erai/clipped_by_month_convert_SST/'
# era_path='/glade/scratch/bkruyt/erai/erai_greatlakes_month/' # not chronological because monthly + buffer
era_path='/glade/scratch/gutmann/icar/forcing/erai_conus/monthly/'  # path with erai files (chronological) cp not corrected!! And CONUS wide...

modelLs       = ['MIROC-ES2L']; i=0
scenarios     = ['ssp370'] #['ssp585'] #

raw_dir = '/glade/scratch/bkruyt/CMIP6/raw/' #MIROC-ES2L/historical' 
in_dir = '/glade/scratch/bkruyt/CMIP6/BC_3D_merged/' #MIROC-ES2L/historical' 
out_dir='/glade/scratch/bkruyt/CMIP6/BC_2D_3D/'


In [22]:
# Load the ERAi data, crop it and correct the cp 

dsObs=xr.open_mfdataset(era_path + 'erai*.nc',combine='by_coords') 


if ('Note' not in dsObs['cp'].attrs.keys() or 
   dsObs['cp'].attrs['Note'] != 'Converted from mm/time step (6 hours) to mm/s by dividing by 21600'):
    print( 'ERAi cp data has not been converted to mm/sec - converting now ....' )
    dsObs['cp']=dsObs['cp']/21600
    dsObs['cp']=dsObs['cp'].assign_attrs(
        {'long_name': 'Convective Precipitation', 
         'units': 'mm/s','Note':'Converted from mm/time step (6 hours) to mm/s by dividing by 21600'}
    )
else:
    print( 'no correction to ERAi cp' )
    print( dsObs['cp'].attrs )
    

ERAi cp data has not been converted to mm/sec - converting now ....


In [23]:
%%time
## Generic BC (fut and past)

for i in range(0,len(modelLs)):
    for scen in scenarios:
        t0 =time.time()
        print(str(i+1)+' '+modelLs[i] + ' ' + scen)

        # Load the 3D bc GCM data, onto which we will add the SST and cp 
        dsFull = xr.open_mfdataset(in_dir + modelLs[i] +'/'+ scen +'/'+modelLs[i] + '*.nc',combine='by_coords').load()

        # Get SST and cp from the original (raw) GCM data:
        dsRaw = xr.open_mfdataset(raw_dir + modelLs[i] +'/'+ scen +'/'+modelLs[i] + '*.nc',combine='by_coords')
        dsCp_in = dsRaw['prec'] #.sel(time=slice(t1,t2)) ## WHY slice??? not generic. 
        dsSST_in = dsRaw['SST'] #.sel(time=slice(t1,t2))
        # dsCp_in=dsCp_in.sel(time=~dsCp_in.get_index("time").duplicated())
        # dsSST_in=dsSST_in.sel(time=~dsSST_in.get_index("time").duplicated())
        print("   GCM data loaded")



        # Need to do a interpolation here with xesmf
        cp_grid_with_bounds   = get_latlon_b_rect(dsCp_in,lon_str='lon',lat_str='lat',lon_dim='lon',lat_dim='lat')
        # sst_grid_with_bounds  = get_latlon_b_rect(dsSST_in,lon_str='lon',lat_str='lat',lon_dim='lon',lat_dim='lat')         # redundant, should be same grid?
        erai_grid_with_bounds = get_latlon_b_rect(dsObs['sst'],lon_str='lon',lat_str='lat',lon_dim='lon',lat_dim='lat')
        regridder = xe.Regridder(cp_grid_with_bounds, erai_grid_with_bounds, 'bilinear') # input grid, gird you want to resample to, method

        dsCp_out = regridder(dsCp_in).load()
        dsSST_out = regridder(dsSST_in).load()

        dsObs['cp']  = dsObs['cp'].load()
        dsObs['sst'] = dsObs['sst'].load()

        # the reference dataset should always be historical:
        if scen=='historical':
            dsCp_ref=dsCp_out.sel(time=slice('1979-09-01','2019-08-31'))
            dsSST_ref=dsSST_out.sel(time=slice('1979-09-01','2019-08-31'))
        else:  # load the historical GCM, regrid, ...
            dsRaw_ref = xr.open_mfdataset(raw_dir + modelLs[i] +'/historical/'+modelLs[i] + '*.nc',combine='by_coords')
            dsCp_ref =  regridder(dsRaw_ref['prec']).sel(time=slice('1979-09-01','2019-08-31')).load()
            dsSST_ref =  regridder(dsRaw_ref['SST']).sel(time=slice('1979-09-01','2019-08-31')).load()
            del dsRaw_ref
          
        dsSST_out=dsSST_out.interpolate_na(dim='time')
        print("   Regridded")

        # Bias Correct Convective Precipitation
        daPrecQm=qm.quantile_mapping_by_group(  dsCp_out,
                                                dsCp_ref, # this should always be hist period?
                                                dsObs['cp'].sel(time=slice('1979-09-01','2019-08-31')),
                                                grouper='time.month',detrend=False,use_ref_data=True,extrapolate='1to1',n_endpoints=50)
        daPrecQm = xr.where(dsCp_out <= 0, 0, daPrecQm)
        daPrecQm.attrs = dsCp_in.attrs
        daCpQm=daPrecQm.to_dataset(name='cp').load()
        # # check/ add attrs?
        # print(daPrecQm.attrs)
        print("   BC cp")


        # Bias Correct Sea Surface Temperature
        daSSTQm=qm.quantile_mapping_by_group(   dsSST_out,
                                                dsSST_ref,
                                                dsObs['sst'].sel(time=slice('1979-09-01','2019-08-31')),
                                                grouper='time.month',detrend=True,use_ref_data=True,extrapolate='1to1',n_endpoints=50)
        daSSTQm.attrs = dsSST_in.attrs
        daSSTQm=daSSTQm.to_dataset(name='tskin').load()
        print("   BC SST")
        print("   total BC time: ", time.time()-t0, "\n") 
        
        if scen == 'historical':
            # time_s = np.arange(1950, 2015, 10)
            # time_f = np.arange(1959, 2099, 10)
            time_s = np.arange(1950, 2015, 10)
            time_f = np.append(np.arange(1959, 2010, 10), 2014)
        else:
            # time_s = np.arange(2015, 2099, 10)
            # time_f = np.arange(2024, 2100, 10)
            time_s = np.append(2015, np.arange(2020, 2099, 10) )
            time_f = np.append(2019, np.arange(2029, 2100, 10) )

        if not os.path.exists(out_dir+'/'+modelLs[i]+'/'+scen):
            os.makedirs(out_dir+'/'+modelLs[i]+'/'+scen)

            
        # TMP:
        time_s=[2095]; time_f=[2099]; t=0
        
        # save the result in 10 year chunks:
        for t in range(0,len(time_s)):
        
            
            t1=time.time()

            dsOut=xr.merge([#dsGCM_bc.sel(time=slice(time_s[t],time_f[t])),
                            dsFull.sel(time=slice(str(time_s[t]),str(time_f[t]))),
                            daCpQm.sel(time=slice(str(time_s[t]),str(time_f[t]))),
                            daSSTQm.sel(time=slice(str(time_s[t]),str(time_f[t])))])

            start_year = str(time_s[t])[0:4]
            end_year   = str(time_f[t])[0:4]
            print('   ', dsOut.time.values.min().astype('datetime64[h]'), dsOut.time.values.max().astype('datetime64[h]') )
            dsOut.to_netcdf(out_dir+'/'+modelLs[i]+'/'+scen+'/' + modelLs[i] + '_bias_corr_final_'+start_year+'-'+end_year+'.nc')
            print('   ', out_dir + modelLs[i]+'_bias_corr_final_'+start_year+'-'+end_year+'.nc')
            print('   merging and writing took ',time.time()-t1 )
               
            del dsOut
 
        print(" \n    - - - - 2d Bias Correction DONE!  - - - - ") 
        print("   total time: ", time.time()-t0, "\n") 

1 MIROC-ES2L ssp370
   GCM data loaded
   Regridded
kwargs: {'alpha': 0.4, 'beta': 0.4, 'extrapolate': '1to1', 'n_endpoints': 50}
putting the trend back
kwargs: {'alpha': 0.4, 'beta': 0.4, 'extrapolate': '1to1', 'n_endpoints': 50}
putting the trend back
kwargs: {'alpha': 0.4, 'beta': 0.4, 'extrapolate': '1to1', 'n_endpoints': 50}
putting the trend back
kwargs: {'alpha': 0.4, 'beta': 0.4, 'extrapolate': '1to1', 'n_endpoints': 50}
putting the trend back
kwargs: {'alpha': 0.4, 'beta': 0.4, 'extrapolate': '1to1', 'n_endpoints': 50}
putting the trend back
kwargs: {'alpha': 0.4, 'beta': 0.4, 'extrapolate': '1to1', 'n_endpoints': 50}
putting the trend back
kwargs: {'alpha': 0.4, 'beta': 0.4, 'extrapolate': '1to1', 'n_endpoints': 50}
putting the trend back
kwargs: {'alpha': 0.4, 'beta': 0.4, 'extrapolate': '1to1', 'n_endpoints': 50}
putting the trend back
kwargs: {'alpha': 0.4, 'beta': 0.4, 'extrapolate': '1to1', 'n_endpoints': 50}
putting the trend back
kwargs: {'alpha': 0.4, 'beta': 0.4, 'ex

### Paralllel attempt... fail

In [17]:


### PArallel version
import multiprocessing as mp

In [11]:
def write_to_file(dsFull, daCpQm, daSSTQm, time_s ):

        t1=time.time()

        time_f=time_s+9

        dsOut=xr.merge([dsFull.sel(time=slice(str(time_s[t]),str(time_f[t]))),
                        daCpQm.sel(time=slice(str(time_s[t]),str(time_f[t]))),
                        daSSTQm.sel(time=slice(str(time_s[t]),str(time_f[t])))])

        start_year = str(time_s[t])[0:4]
        end_year   = str(time_f[t])[0:4]
        print('   ', dsOut.time.values.min().astype('datetime64[h]'), dsOut.time.values.max().astype('datetime64[h]') )
        dsOut.to_netcdf(out_dir + modelLs[i] + '_bias_corr_final_'+start_year+'-'+end_year+'.nc')
        print('   ', out_dir + modelLs[i]+'_bias_corr_final_'+start_year+'-'+end_year+'.nc')
        print('   merging and writing took ',time.time()-t1 )

In [14]:
time_s[0]

2015

In [16]:

Nprocs=4 #len(time_s) # abit much maybe

## Call in parallel :
with mp.Pool(processes = Nprocs) as p:
    [p.apply(write_to_file, args=(dsFull, daCpQm, daSSTQm, time_start )) for time_start in time_s] 




OverflowError: cannot serialize a bytes object larger than 4 GiB

In [ ]:
%%time
## Generic BC (fut and past)

for i in range(0,len(modelLs)):
    for scen in scenarios:
        t0 =time.time()
        print(str(i+1)+' '+modelLs[i] + ' ' + scen)

        # Load the 3D bc GCM data, onto which we will add the SST and cp 
        dsFull = xr.open_mfdataset(in_dir + modelLs[i] +'/'+ scen +'/'+modelLs[i] + '*.nc',combine='by_coords').load()

        # Get SST and cp from the original (raw) GCM data:
        dsRaw = xr.open_mfdataset(raw_dir + modelLs[i] +'/'+ scen +'/'+modelLs[i] + '*.nc',combine='by_coords')
        dsCp_in = dsRaw['prec'] #.sel(time=slice(t1,t2)) ## WHY slice??? not generic. 
        dsSST_in = dsRaw['SST'] #.sel(time=slice(t1,t2))
        # dsCp_in=dsCp_in.sel(time=~dsCp_in.get_index("time").duplicated())
        # dsSST_in=dsSST_in.sel(time=~dsSST_in.get_index("time").duplicated())
        print("   GCM data loaded")



        # Need to do a interpolation here with xesmf
        cp_grid_with_bounds   = get_latlon_b_rect(dsCp_in,lon_str='lon',lat_str='lat',lon_dim='lon',lat_dim='lat')
        # sst_grid_with_bounds  = get_latlon_b_rect(dsSST_in,lon_str='lon',lat_str='lat',lon_dim='lon',lat_dim='lat')         # redundant, should be same grid?
        erai_grid_with_bounds = get_latlon_b_rect(dsObs['sst'],lon_str='lon',lat_str='lat',lon_dim='lon',lat_dim='lat')
        regridder = xe.Regridder(cp_grid_with_bounds, erai_grid_with_bounds, 'bilinear') # input grid, gird you want to resample to, method

        dsCp_out = regridder(dsCp_in).load()
        dsSST_out = regridder(dsSST_in).load()

        dsObs['cp']  = dsObs['cp'].load()
        dsObs['sst'] = dsObs['sst'].load()

        # the reference dataset should always be historical:
        if scen=='historical':
            dsCp_ref=dsCp_out.sel(time=slice('1979-09-01','2019-08-31'))
            dsSST_ref=dsSST_out.sel(time=slice('1979-09-01','2019-08-31'))
        else:  # load the historical GCM, regrid, ...
            dsRaw_ref = xr.open_mfdataset(raw_dir + modelLs[i] +'/historical/'+modelLs[i] + '*.nc',combine='by_coords')
            dsCp_ref =  regridder(dsRaw_ref['prec']).sel(time=slice('1979-09-01','2019-08-31')).load()
            dsSST_ref =  regridder(dsRaw_ref['SST']).sel(time=slice('1979-09-01','2019-08-31')).load()
          
        dsSST_out=dsSST_out.interpolate_na(dim='time')
        print("   Regridded")

        # Bias Correct Convective Precipitation
        daPrecQm=qm.quantile_mapping_by_group(  dsCp_out,
                                                dsCp_ref, # this should always be hist period?
                                                dsObs['cp'].sel(time=slice('1979-09-01','2019-08-31')),
                                                grouper='time.month',detrend=False,use_ref_data=True,extrapolate='1to1',n_endpoints=50)
        daPrecQm = xr.where(dsCp_out <= 0, 0, daPrecQm)
        daPrecQm.attrs = dsCp_in.attrs
        daCpQm=daPrecQm.to_dataset(name='cp').load()
        # check/ add attrs?
        print(daPrecQm.attrs)
        print("   BC cp")


        # Bias Correct Sea Surface Temperature
        daSSTQm=qm.quantile_mapping_by_group(   dsSST_out,
                                                dsSST_ref,
                                                dsObs['sst'].sel(time=slice('1979-09-01','2019-08-31')),
                                                grouper='time.month',detrend=True,use_ref_data=True,extrapolate='1to1',n_endpoints=50)
        daSSTQm.attrs = dsSST_in.attrs
        daSSTQm=daSSTQm.to_dataset(name='tskin').load()
        print("   BC SST")
        print("   total BC time: ", time.time()-t0, "\n") 
        
        if scen == 'historical':
            time_s = np.arange(1950, 2015, 10)
            time_f = np.arange(1959, 2099, 10)
        else:
            time_s = np.arange(2015, 2099, 10)
            time_f = np.arange(2024, 2100, 10)

        if not os.path.exists(out_dir):
            os.makedirs(out_dir)


        Nprocs=4 #len(time_s) # abit much maybe
   
        ## Call in parallel :
        with mp.Pool(processes = Nprocs) as p:
            [p.apply(write_to_file, args=(dsFull, daCpQm, daSSTQm, time_start )) for time_start in time_s] 


            

            
#             # save the result in 10 year chunks:
#         for t in range(0,len(time_s)):
           
#             t1=time.time()

#             dsOut=xr.merge([#dsGCM_bc.sel(time=slice(time_s[t],time_f[t])),
#                             dsFull.sel(time=slice(str(time_s[t]),str(time_f[t]))),
#                             daCpQm.sel(time=slice(str(time_s[t]),str(time_f[t]))),
#                             daSSTQm.sel(time=slice(str(time_s[t]),str(time_f[t])))])

#             start_year = str(time_s[t])[0:4]
#             end_year   = str(time_f[t])[0:4]
#             print('   ', dsOut.time.values.min().astype('datetime64[h]'), dsOut.time.values.max().astype('datetime64[h]') )
#             dsOut.to_netcdf(out_dir + modelLs[i] + '_bias_corr_final_'+start_year+'-'+end_year+'.nc')
#             print('   ', out_dir + modelLs[i]+'_bias_corr_final_'+start_year+'-'+end_year+'.nc')
#             print('   merging and writing took ',time.time()-t1 )
                  
 

### Original below

In [ ]:

for i in range(0,len(modelLs)):
    
    print(str(i+1)+' '+modelLs[i])

    # print(modelLs[i]+' ssp585')
    
    # Convective Precipitation
    # dsFull = xr.open_mfdataset(in_dir + modelLs[i] + '*.nc',combine='by_coords').load()
    dsFull = xr.open_mfdataset(in_dir + modelLs[i] +'/'+ scen +'/'+modelLs[i] + '*.nc',combine='by_coords').load()
    dsCp = dsFull['prec'].sel(time=slice(t1,t2))

    # Sea Surface Temperature
    dsGCM_b4_bc = dsFull['SST'].sel(time=slice(t1,t2))
    dsGCM_b4_bc=dsGCM_b4_bc.sel(time=~dsGCM_b4_bc.get_index("time").duplicated())
    print("Raw data loaded")

    # To store the output data
    files=sorted(glob.glob('/glade/scratch/abby/ESM_bias_correction/ref_period_only/bias_corrected_merged_' + modelLs[i] + '*.nc'))
    dsGCM_bc=xr.open_mfdataset(files,combine='by_coords').sel(time=slice(t1,t2))
    dsGCM_bc=dsGCM_bc.sel(time=~dsGCM_bc.get_index("time").duplicated()) #remove any possible duplicate value
    print("BC data loaded")
    
    # Need to do a interpolation here with xesmf
    cp_grid_with_bounds   = get_latlon_b_rect(dsCp,lon_str='lon',lat_str='lat',lon_dim='lon',lat_dim='lat')
    sst_grid_with_bounds  = get_latlon_b_rect(dsGCM_b4_bc,lon_str='lon',lat_str='lat',lon_dim='lon',lat_dim='lat')
    erai_grid_with_bounds = get_latlon_b_rect(dsObs['sst'],lon_str='lon',lat_str='lat',lon_dim='lon',lat_dim='lat')

    regridder = xe.Regridder(cp_grid_with_bounds, erai_grid_with_bounds, 'bilinear') # input grid, gird you want to resample to, method
    dsCp_out = regridder(dsCp)

    regridder = xe.Regridder(sst_grid_with_bounds, erai_grid_with_bounds, 'bilinear') # input grid, gird you want to resample to, method
    dsSST_out = regridder(dsGCM_b4_bc)

    dsObs['cp']  = dsObs['cp'].load()
    dsCp_out     = dsCp_out.load()
    dsObs['sst'] = dsObs['sst'].load()
    dsSST_out    = dsSST_out.load()
    
    dsSST_out=dsSST_out.interpolate_na(dim='time')
    print("Regridded")
    # Bias Correct Convective Precipitation
    
    # add n endpoints = 50 to both
    daPrecQm=qm.quantile_mapping_by_group(dsCp_out,
                                                dsCp_out.sel(time=slice('1979-09-01','2019-08-31')),
                                                dsObs['cp'].sel(time=slice('1979-09-01','2019-08-31')),
                                                grouper='time.month',detrend=False,use_ref_data=True,extrapolate='1to1',n_endpoints=50)
    daPrecQm = xr.where(dsCp_out <= 0, 0, daPrecQm)
    daCpQm=daPrecQm.to_dataset(name='cp')
    print("BC cp")


    # Bias Correct Sea Surface Temperature
    daSSTQm=qm.quantile_mapping_by_group(dsSST_out,
                                                dsSST_out.sel(time=slice('1979-09-01','2019-08-31')),
                                                dsObs['sst'].sel(time=slice('1979-09-01','2019-08-31')),
                                                grouper='time.month',detrend=True,use_ref_data=True,extrapolate='1to1',n_endpoints=50)
    daSSTQm=daSSTQm.to_dataset(name='tskin')
    print("BC SST")


    time_s=['1950-01-01T12:00','1960-01-01T00:00','1970-01-01T00:00','1980-01-01T00:00','1990-01-01T00:00',
            '2000-01-01T00:00','2010-01-01T00:00']

    time_f=['1959-12-31T18:00','1969-12-31T18:00','1979-12-31T18:00','1989-12-31T18:00','1999-12-31T18:00',
            '2009-12-31T18:00','2019-08-31T18:00']


    for t in range(0,len(time_s)):




        import os
        if not os.path.exists(out_dir):
            os.makedirs(out_dir)

        dsOut=xr.merge([dsGCM_bc.sel(time=slice(time_s[t],time_f[t])),
                        daCpQm.sel(time=slice(time_s[t],time_f[t])),
                        daSSTQm.sel(time=slice(time_s[t],time_f[t]))])

        start_year = time_s[t][0:4]
        end_year   = time_f[t][0:4]
        print(out_dir+'/'+modelLs[i]+'_bias_corr_final_'+start_year+'-'+end_year+'.nc')
        dsOut.to_netcdf(out_dir+'/'+modelLs[i]+'_bias_corr_final_'+start_year+'-'+end_year+'.nc')

